##### Exercício 14.1.
Escreva uma função chamada sed que receba como argumentos uma string padrão, uma string de substituicao e dois nomes de arquivos. Ela deve ler o conteúdo do primeiro, armazenar no segundo (criando-o se necessário) e substituir a string padrão pela string de substituicao

In [ ]:
# import time

import re

def sed(string: str, replace: str, filename: str, new_file: str) -> None:

    # start = time.time()
    try:
        with open(filename, mode='r', encoding='utf-8') as file:
            text = file.read()
        
        print(f'{filename} read sucessufully.')

    except FileNotFoundError as e:
        print(f'{e}')

    # text to be write in new text file
    new_text = re.sub(pattern=string, repl=replace, string=text)

    # opening a new text file to write the content
    with open('../new_file.txt', mode='w') as file:
        file.write(new_text)
    
    print(f'{new_file} exported sucessufully.')

    return

def main():
    
    folder = '../'
    
    str_input = 'bee'
    str_replace = 'cat'
    filename = folder + 'poetry_cap14.txt'
    new_filename = folder + 'new_file.txt'

    sed(string=str_input, replace=str_replace, filename=filename, new_file=new_filename)

main()

../poetry_cap14.txt read sucessufully.
../new_file.txt exported sucessufully.


##### Exercício 14.2
Escreva um programa que importe o módulo que mapeia uma string à seus anagramas (disponível em https://github.com/AllenDowney/ThinkPython2/blob/master/code/anagram_sets.py) e acrescente duas novas funções: uma que armazene esse hash em um db e outra que faz a busca de uma palavra e devolva seus anagramas.

O programa abaixo usa o dbm.dumb, usa uma versão mais lenta e simples, pois não estamos usando os módulos dbm.sqlite3, dbm.gnu ou dbm.ndbm. Logo, para hashs muito grandes, a performance do programa é ruim.

In [22]:
import dbm

def store_anagrams(db_name: str) -> None:
    """Open a database for read and write and creates a new one if it doesn't exist"""
    
    # map all words in words.txt to their anagrams
    hash = {
    'aa': ['aa'],
    'aah': ['aah', 'aha'],
    'aadeh': ['aahed', 'ahead'],
    'aaghin': ['aahing'],
    'aahs': ['aahs']
    }

    # db = dbm.open(db_name, 'c') # open the database for read and write and create a new one if it doesn't exist
    
    with dbm.open(db_name, 'c') as db: # open the database for read and write and create a new one if it doesn't exist

        # inserting the keys and values in the database
        for k, v in hash.items():
            db[k] = ', '.join(v)

    return 


def read_anagrams(db_name: str, word: str) -> list:
    """Open a existing database in read mode and get the value in key 'word'"""

    with dbm.open(db_name, 'r') as db:

        # get the value in key 'searching_word' (which is stored as bytes), convert it to string and split it in a list
        try:
            word_anagrams = str(db.get(searching_word), encoding='utf-8')
            list_anagrams = word_anagrams.split(', ')
        
        except TypeError:
            list_anagrams = []

    return list_anagrams

database = '../db_AnagramsCap14'
searching_word = 'aadeh'

# store anagrams
store_anagrams(database)
result = read_anagrams(db_name=database, word=searching_word)

print(f"for word {searching_word} we have the following anagrams: {result}")


for word aadeh we have the following anagrams: ['aahed', 'ahead']


##### Exercício 14.3
Escreva um programa que procure um diretório, retorne todos os arquivos dentro dele e procure por arquivos duplicados

In [96]:
import os
import hashlib

def walk(dirname: str) -> None:
    """Prints the names of all files in dirname and its subdirectories"""

    file_list = []

    for name in os.listdir(dirname): # walk through all files and folders in directory
        
        path = os.path.join(dirname, name) # join the directory name with the file name

        if os.path.isfile(path): # check if the path is a file
            print(path)
        
        else:                    # if it is a directory call the function again
            walk(path)

    return file_list

# walk through the above directory
folder = '..'

walk(folder)

..\.git\COMMIT_EDITMSG
..\.git\config
..\.git\description
..\.git\HEAD
..\.git\hooks\applypatch-msg.sample
..\.git\hooks\commit-msg.sample
..\.git\hooks\fsmonitor-watchman.sample
..\.git\hooks\post-update.sample
..\.git\hooks\pre-applypatch.sample
..\.git\hooks\pre-commit.sample
..\.git\hooks\pre-merge-commit.sample
..\.git\hooks\pre-push.sample
..\.git\hooks\pre-rebase.sample
..\.git\hooks\pre-receive.sample
..\.git\hooks\prepare-commit-msg.sample
..\.git\hooks\push-to-checkout.sample
..\.git\hooks\sendemail-validate.sample
..\.git\hooks\update.sample
..\.git\index
..\.git\info\exclude
..\.git\logs\HEAD
..\.git\logs\refs\heads\main
..\.git\logs\refs\remotes\origin\main
..\.git\objects\09\b50e1ad77fd12d98364eb81861e1c1e7c644d5
..\.git\objects\0a\48ad0c0fdd22c39a6e614b958f409cdbc0f14b
..\.git\objects\0c\9c91d30b547a5f4ec7d14a26abf37fabec41a7
..\.git\objects\0f\7785f6d55b55141bb85b71fcc8e9555f0e2a14
..\.git\objects\11\76aba5ce05cf008f027e92b961f6f71a88fd76
..\.git\objects\14\299e0febed66

[]

Adaptando a função walk para obter os valores hash dos arquivos (soma md5) e identificar as duplicatas

In [ ]:
import os
import hashlib
from collections import defaultdict

def calculate_file_hash(filename: str):
    """Computes the MD5 checksum of the contents of a file"""

    hash = hashlib.md5() # create a new hash object

    with open(filename, 'rb') as file: # open the file in read mode and binary mode to avoid encoding problems

        while chunk := file.read(8192): # read the file in chunks of 8192 bytes using walrus operator
            hash.update(chunk)
    
    return hash.hexdigest()

def get_files(dirname: str) -> list:
    """get names of all files in dirname and its subdirectories"""

    files_list = [] # create a list to store the files

    for name in os.listdir(dirname): # walk through all files and folders in directory
        
        path = os.path.join(dirname, name) # join the directory name with the file name

        if os.path.isfile(path): # check if the path is a file
        
            files_list.append(path)

    return files_list

def identify_duplicates(dirname: str) -> dict[str]:
    """Prints the names of all files in dirname and its subdirectories"""

    dict_hash_values = defaultdict(list) # set a default value for the dictionary

    dir_files = get_files(dirname) # get the list of files in the directory

    for path in dir_files: # walk through all files in the directory
            
        hash_value = calculate_file_hash(path) # calculate the hash value of the file

        dict_hash_values[hash_value].append(path) # append the filename in the dictionary  

    return dict_hash_values

def remove_duplicates(dict_hash_values: dict[str]) -> None:
    """Remove the duplicate files"""

    for file_list in dict_hash_values.values(): # traverses the dictionary values, which are lists of files
        
        if len(file_list) > 1: # check if the list has more than one file
            print(f'Duplicate files: {file_list}\n')

            try:    
                for file in file_list[1:]: # traverses the list of files of each hash value, starting from index 1 to keep one file and remove the copies
                    os.remove(file) # remove the files
                    
                    print(f'{file} removed sucessfully.\n')

            except FileNotFoundError as e:
                print(f'{e}')


# walk through the above directory
folder = '../'

# get the dictionary with the duplicate files
duplicates = identify_duplicates(folder)

# remove the duplicate files
remove_duplicates(duplicates)

No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.
No duplicates found.


In [85]:
def get_files(dirname: str) -> list:
    """get names of all files in dirname and its subdirectories"""

    files_list = [] # create a list to store the files

    for name in os.listdir(dirname): # walk through all files and folders in directory
        
        path = os.path.join(dirname, name) # join the directory name with the file name

        if os.path.isfile(path): # check if the path is a file
        
            files_list.append(path)
            # print(path)

    return files_list

# walk through the above directory
folder = '..'

files = get_files(folder)

# with open (files[10], 'r') as file:
#     text = file.read()

# text

files

['..\\.gitignore',
 '..\\.python-version',
 '..\\db_AnagramsCap14.bak',
 '..\\db_AnagramsCap14.dat',
 '..\\db_AnagramsCap14.dir',
 '..\\emma.txt',
 '..\\file_duplicate01.txt',
 '..\\file_duplicate02.txt',
 '..\\hello.py',
 '..\\new_file.txt',
 '..\\poetry_cap14.txt',
 '..\\pyproject.toml',
 '..\\README.md',
 '..\\requirements.txt',
 '..\\uv.lock',
 '..\\words.txt']

##### Walrus Operator

In [18]:
walrus = False

In [17]:
(walrus := True)

True

In [19]:
soma = 1 + 2

In [20]:
(soma:= 1 + 2)

3

In [ ]:
# walryus operator use case 1
numbers_list = [2, 8, 0, 1, 1, 9, 7, 7]

descr = {
    'length': len(numbers_list),
    'sum': sum(numbers_list),
    'mean': sum(numbers_list) / len(numbers_list) # we called the sum() and len() functions twice, which is not efficient
}

descr

{'length': 8, 'sum': 35, 'mean': 4.375}

In [ ]:
# we can avoid calling the sum() and len() functions twice by using assignment statements outside the dictionary

numbers_list = [2, 8, 0, 1, 1, 9, 7, 7]

len_list = len(numbers_list)
sum_list = sum(numbers_list)

descr = {
    'length': len_list,
    'sum': sum_list,
    'mean': sum_list / len_list 
}

descr

{'length': 8, 'sum': 35, 'mean': 4.375}

In [24]:
# we can use the walrus operator to avoid creating two variables and make the code more readable
# these two variables  aree used only to optimize the code and with the walrus operator we can make this role clearer

numbers_list = [2, 8, 0, 1, 1, 9, 7, 7]

descr = {
    'length': (len_list := len(numbers_list)),
    'sum': (sum_list := sum(numbers_list)),
    'mean': sum_list / len_list 
}

descr

{'length': 8, 'sum': 35, 'mean': 4.375}

In [ ]:
# walrus operator use case 2: list comprehensions

def func(x):
    return x ** 2

# we wanna create a list with the squares of the numbers in numbers_list and afteer that we wanna filter only the numbers that are greater than 10
numbers_list = [2, -8, 0, -1, 1, -9, 7, -7]

# without walrus operator: we called func() twice
list_abs = [func(num) for num in numbers_list if func(num) > 10]
list_abs

# with the walrus operator: we called func() only once
# func is evaluated and assigned to num_pow variable and we can use it in the list comprehension
list_absv2 = [num_pow for num in numbers_list if (num_pow := func(num)) > 10] 
list_absv2

print(f'Without walrus operator: {list_abs}\nWith walrus operator: {list_absv2}')

Without walrus operator: [64, 81, 49, 49]
With walrus operator: [64, 81, 49, 49]


In [ ]:
# walrus operator use case 3: while loop

# now we want to check if the user input is valid 

valid_inputs = ['yes', 'no', 'y', 'n']

# way 1 without walrus operator
user_input = input('Enter your answer: ')

while user_input not in valid_inputs:
    print(f'Invalid input. Please enter one of the following: {valid_inputs}\n')
    user_input = input('Enter your answer: ') # we called input() twice, outside and inside the while loop

print(f'Your input is correct: {user_input}')

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Your input is correct: y


In [29]:
# way 2 without walrus operator
# we can use while True and break the loop when the user input is valid

while True:
    user_input = input('Enter your answer: ')
    
    if user_input in valid_inputs:
        break
    
    print(f'Invalid input. Please enter one of the following: {valid_inputs}\n')

print(f'Your input is correct: {user_input}')

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Your input is correct: y


In [31]:
# way 3: with walrus operator

while (user_input := input('Enter your answer: ')) not in valid_inputs:
    print(f'Invalid input. Please enter one of the following: {valid_inputs}\n')

print(f'Your input is correct: {user_input}')    

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Invalid input. Please enter one of the following: ['yes', 'no', 'y', 'n']

Your input is correct: y


In [ ]:
# we can also use walrus operator to find witnesses and counterexamples for any() and all() functions

# any() function returns True if any element of the iterable is true
# but we can use the walrus operator to find the first element that is True

cities = ["Vancouver", "Oslo", "Berlin", "Krakow", "Graz", "Belgrade"]

# example: we wanna find the first city that starts with 'B'
if any((witness := city).startswith('B') for city in cities):
    print(f'The first city that starts with "B" is: {witness}\n')

# trying to find if all cities have more than 10 characters
if not all(len(counterexample := city) > 10 for city in cities):
    print(f'The first city with less than 10 characters is: {counterexample}')

The first city that starts with "B" is: Berlin

The first city with less than 10 characters is: Vancouver
